# Atividade 2 - Etapa 2 - Tokenização com Word2Vec

Tokenização do dataset de notícias(Folha UOL) utilizando o modelo Word2Vec

In [ ]:
# importação do dataset
import pandas as pd

df = pd.read_csv('dataset/articles.csv', encoding='utf8')
print('linhas, colunas):', df.shape)

df.head()

### **Ajustes para aprendizado - já realizados na Etapa1_v02**

In [ ]:
df.drop_duplicates(subset=['text'], inplace=True)
df.drop(['subcategory', 'link', 'date'], axis=1, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

In [ ]:
df['category'].value_counts()

### **Utilizarei as 7 categorias mais relevantes no treinamento a fim de deixar o dataset mais balanceado**

In [ ]:
df = df[df['category'].isin(list(df['category'].value_counts()[:7].index))]

df['category'] = df['category'].replace('guia-de-livros-discos-filmes', 'guia-de-livros-filmes-discos')
alvo, valores = pd.factorize(df['category'], sort=True)
df['target'] = alvo
print('Novo tamanho do dataset:', len(df))
list(enumerate(valores))

In [ ]:
import nltk
import tensorflow as tf
import re, string, unicodedata
import matplotlib.pyplot as plt

from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
from wordcloud import WordCloud, STOPWORDS

#instalação do modelo 'stopwords'
nltk.download('stopwords')

from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing import text, sequence
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

**Remoção de Palavras Irrelevantes (Stopwords)**

Não acrescentam muito a sentença e podem ser facilmente ignoradas sem sacrificar o significado do texto.

In [ ]:
stop = set(stopwords.words('portuguese'))
punctuation = list(string.punctuation)
stop.update(punctuation)

### Limpeza de Texto

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('', '', text)

# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

#Removing the noisy text
def denoise_text(text):
    #text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

#Apply function on review column
df['text']=df['text'].apply(denoise_text)

**WORDCLOUD**

In [ ]:
# criação de uma nuvem de palavras para textos da categoria 'poder'
plt.figure(figsize = (10, 10)) # Text that is Not Sarcastic
wc = WordCloud(max_words=2000 , width=1600 , height=800).generate(" ".join(df[df.category == 'poder'].text))
plt.imshow(wc , interpolation = 'bilinear')

### **Introdução à incorporação de palavras e Word2Vec**

Representação do vocabulário de documentos. É capaz de de capturar o contexto de uma palavra em um documento, semelhança semântica e sintática, relação com outras palavras.

Incorporação de palavras: representações vetoriais de uma palavra específica. Word2Vec é uma técnica para aprender incorporações de palavras usando rede neural superficial.

**Convertendo para um formato aceitado pela biblioteca _gensim_**

In [ ]:
words = []
for i in df.text.values:
    # se não for string, converte para string
    if not isinstance(i, str):
        i = str(i)
    words.append(i.split())

words[:2]

In [ ]:
import gensim
# dimensão do vetor a ser gerado
EMBEDDING_DIM = 100
# criando um vetor de palavras através do Método Word2Vec 
w2v_model = gensim.models.Word2Vec(sentences=words, vector_size=EMBEDDING_DIM, window=5, min_count=1)

In [ ]:
len(w2v_model.wv.key_to_index)

In [ ]:
tokenizer = text.Tokenizer(num_words=35000)
tokenizer.fit_on_texts(words)
tokenized_train = tokenizer.texts_to_sequences(words)
x = sequence.pad_sequences(tokenized_train , maxlen=20)